# Практическое задание

## Данные о студенте

1. **ФИО**: Кираев Булат Рустамович
2. **Факультет**: мех-мат
3. **Курс**: 3
4. **Группа**: 331

## Замечания

* Заполненный ноутбук необходимо сдать боту
* Соблюдаем кодекс чести (по нулям и списавшему, и давшему списать)
* Можно (и нужно!) применять для реализации только библиотеку **Numpy**
* Ничего, крому Numpy, нельзя использовать для реализации 
* **Keras** используется только для тестирования Вашей реализации
* Если какой-то из классов не проходит приведенные тесты, то соответствующее задание не оценивается
* Возможно использование дополнительных (приватных) тестов
 

## Реализация собственного нейросетевого пакета для запуска и обучения нейронных сетей

Задание состоит из трёх частей:
1. Реализация прямого вывода нейронной сети (5 баллов)
2. Реализация градиентов по входу и распространения градиента по сети (5 баллов)
3. Реализация градиентов по параметрам и метода обратного распространения ошибки с обновлением парметров сети (10 баллов)

Дополнительные баллы можно получить при реализации обучения сети со свёрточными слоями (10 баллов), с транспонированной свёрткой (10 баллов), дополнительного оптимизатора (5 баллов). 

###  1. Реализация вывода собственной нейронной сети

1.1 Внимательно ознакомьтесь с интерфейсом слоя. Любой слой должен содержать как минимум три метода:
- конструктор
- прямой вывод 
- обратный вывод, производные по входу и по параметрам

In [405]:
class Layer(object):
    def __init__(self):
        self.name = 'Layer'       
    def forward(self, input_data):
        pass
    def backward(self, input_data):
        return [self.grad_x(input_data), self.grad_param(input_data)]
    
    def grad_x(self, input_data):
        pass
    def grad_param(self, input_data):
        return []
    
    def update_param(self, grads, learning_rate):
        pass


1.2 Ниже предствален интерфейс класса  Network. Обратите внимание на реализацию метода predict, который последовательно обрабатывает входные данные слой за слоем.

In [406]:
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

class Network(object):
    def __init__(self, layers, loss=None):
        self.name = 'Network'
        self.layers = layers
        self.loss = loss
    
    def forward(self, input_data):
        return self.predict(input_data)
    
    def grad_x(self, input_data, labels):
        lrs = []
        for lr in self.layers:
            lrs.append(np.transpose(lr.grad_x(input_data), (0, 2, 1)))
            input_data = lr.forward(input_data)
        answ = np.copy(lrs[0])
        a = [0 for i in range(input_data.shape[0])]
        for i in range(1, len(lrs)):
            for j in range(input_data.shape[0]):
                a[j] = np.matmul(answ[j, :, :], lrs[i][j, :, :])
            answ = np.asarray(a)
        L_y = self.loss.grad_x(input_data, labels)
        for j in range(input_data.shape[0]):
            a[j] = np.matmul(answ[j, :, :], L_y[j, :])
        return np.asarray(a)
    
    def grad_param(self, input_data, labels):
        out = []
        for layer in self.layers:
            out.append(layer.grad_param(input_data))
            input_data = layer.forward(input_data)
        return out

    def update(self, grad_list, learning_rate):
        for i in range(len(grad_list)):
            self.layers[i].update_param(grad_list[i], learning_rate)

    def predict(self, input_data):
        current_input = input_data
        for layer in self.layers:
            current_input = layer.forward(current_input)     
        return current_input
    
    def calculate_loss(self, input_data, labels):
        return self.loss.forward(self.predict(input_data), labels)
    
    def train_step(self, input_data, labels, learning_rate=0.001):
        grad_list = self.grad_param(input_data, labels)
        self.update(grad_list, learning_rate)
    
    
    def fit(self, trainX, trainY, validation_split=0.25, 
            batch_size=1, nb_epoch=1, learning_rate=0.01):
        
        train_x, val_x, train_y, val_y = train_test_split(trainX, trainY, 
                                                          test_size=validation_split,
                                                          random_state=42)
        for epoch in range(nb_epoch):
            #train one epoch
            for i in tqdm(range(int(len(train_x)/batch_size))):
                batch_x = train_x[i*batch_size: (i+1)*batch_size]
                batch_y = train_y[i*batch_size: (i+1)*batch_size]
                self.train_step(batch_x, batch_y, learning_rate)
            #validate
            val_accuracy = self.evaluate(val_x, val_y)
            print('%d epoch: val %.2f' %(epoch+1, val_accuracy))
            
    def evaluate(self, testX, testY):
        y_pred = np.argmax(self.predict(testX), axis=1)            
        y_true = np.argmax(testY, axis=1)
        val_accuracy = np.sum((y_pred == y_true))/(len(y_true))
        return val_accuracy

#### 1.1 Необходимо реализовать метод forward для вычисления следующих слоёв:

- DenseLayer
- ReLU
- Softmax
- FlattenLayer
- MaxPooling

In [407]:
#импорты
import numpy as np

In [427]:
class DenseLayer(Layer):
    def __init__(self, input_dim, output_dim, W_init=None, b_init=None):
        self.name = 'Dense'
        self.input_dim = input_dim
        self.output_dim = output_dim
        if W_init is None or b_init is None:
            self.W = np.random.random((input_dim, output_dim))
            self.b = np.zeros(output_dim, 'float32')
        else:
            self.W = W_init
            self.b = b_init
    def forward(self, input_data):
        out = np.zeros((input_data.shape[0], self.output_dim))
        for i in range(input_data.shape[0]):
            for j in range(self.output_dim):
                out[i, j] = np.dot(input_data[i, :], self.W[:, j]) + self.b[j]
        return out
    def grad_x(self, input_data):
        out = np.zeros((input_data.shape[0], self.output_dim, self.input_dim))
        for i in range(out.shape[0]):
            out[i, :, :] = self.W.transpose()
        return(out)
    def grad_b(self, input_data):
        out = np.zeros((input_data.shape[0], self.output_dim, self.output_dim))
        for i in range(out.shape[0]):
            out[i, :, :] = np.eye(self.output_dim, self.output_dim)
        return out
    def grad_W(self, input_data):
        out = np.zeros((input_data.shape[0], self.output_dim, self.input_dim))
        for i in range(out.shape[0]):
            for j in range(out.shape[1]):
                out[i, j, :] = input_data[i, :]
        return out
    
    def update_W(self, grad, learning_rate):
        self.W -= learning_rate * np.mean(grad, axis=0).reshape(self.W.shape)
    
    def update_b(self, grad,  learning_rate):
        self.b -= learning_rate * np.sum(np.mean(grad, axis=0), axis=0)
        
    def update_param(self, params_grad, learning_rate):
        self.update_W(params_grad[0], learning_rate)
        self.update_b(params_grad[1], learning_rate)
    
    def grad_param(self, input_data):
        return [self.grad_W(input_data), self.grad_b(input_data)]
    
class ReLU(Layer):
    def __init__(self):
        self.name = 'ReLU'
    def forward(self, input_data):
        self.input_data = input_data
        return np.maximum(0, input_data)
    def grad_x(self, input_data):
        if len(input_data.shape) == 2:
            out = np.zeros((input_data.shape[0], input_data.shape[1], input_data.shape[1]))
            for i in range(out.shape[0]):
                for j in range(out.shape[1]):
                    for k in range(out.shape[1]):
                        if j == k and input_data[i, j] > 0: out[i, j, k] = 1
                        else: out[i, j, k] = 0
        elif len(input_data.shape) == 4:
            shp = input_data.shape[1] * input_data.shape[2] * input_data.shape[3]
            out = np.zeros((input_data.shape[0], shp, shp))
            mas = FlattenLayer.forward(FlattenLayer, input_data)
            for i in range(out.shape[0]):
                for j in range(out.shape[1]):
                    for k in range(out.shape[1]):
                        if j == k and mas[i, j] > 0: out[i, j, k] = 1
                        else: out[i, j, k] = 0
        return out

class Softmax(Layer):
    def __init__(self):
        self.name = 'Softmax'
    def forward(self, input_data):
        exps = np.exp(input_data - np.max(input_data, axis=1, keepdims=True))
        self.output = exps / np.sum(exps, axis=1, keepdims=True)
        return self.output
    def grad_x(self, input_data):
        self.forward(input_data)
        
        batch_size = input_data.shape[0]
        num_classes = self.output.shape[1]
        gradients = np.zeros((batch_size, num_classes, num_classes))
        
        for i in range(batch_size):
            for j in range(num_classes):
                for k in range(num_classes):
                    if j == k:
                        gradients[i, j, k] = self.output[i, j] * (1 - self.output[i, k])
                    else:
                        gradients[i, j, k] = -self.output[i, j] * self.output[i, k]
        
        return gradients

class FlattenLayer(Layer):
    def __init__(self):
        self.name = 'Flatten'
    def forward(self, input_data):
        output = input_data.reshape(input_data.shape[0], -1)
        return output
    def grad_x(self):
        pass

    
class MaxPooling(Layer):
    def __init__(self, pool_size=(2, 2), strides=(2, 2)):
        self.name = 'MaxPooling'
        self.pool_size = pool_size
        self.strides = strides
    
    def forward(self, input_data):
        self.input_shape = input_data.shape
        batch_size, channels, height, width = self.input_shape
        pool_height, pool_width = self.pool_size
        stride_height, stride_width = self.strides
        
        out_height = int((height - pool_height) / stride_height) + 1
        out_width = int((width - pool_width) / stride_width) + 1
        
        output = np.zeros((batch_size, channels, out_height, out_width))
        
        for i in range(out_height):
            for j in range(out_width):
                h_start = i * stride_height
                h_end = h_start + pool_height
                w_start = j * stride_width
                w_end = w_start + pool_width
                output[:, :, i, j] = np.max(input_data[:, :, h_start:h_end, w_start:w_end], axis=(2, 3))
        
        return output

    def grad_x(self):
        pass

#### 1.2 Реализуйте теперь свёрточный слой и транспонированную свёртку  (опционально)

In [409]:
class Conv2DLayer(Layer):
    def __init__(self, kernel_size=3, input_channels=2, output_channels=3, 
                 padding='same', stride=1, K_init=None, b_init=None):
        # padding: 'same' или 'valid'
        # Работаем с квадратными ядрами, поэтому kernel_size - одно число
        # Работаем с единообразным сдвигом, поэтому stride - одно число
        # Фильтр размерности [kernel_size, kernel_size, input_channels, output_channels]
        self.name = 'Conv2D'
        self.kernel_size = kernel_size
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.kernel = K_init
        self.bias = b_init
        self.padding = padding
        self.stride = stride
    def forward(self, input_data):
        # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
        # Вначале нужно проверить на согласование размерностей входных данных и ядра!
        # Нужно заполнить Numpy-тензор out 
        batch, channels, height, width = input_data.shape
        
        if channels != self.input_channels:
            raise ValueError("Input channel dimension does not match kernel's channel dimension.")
        
        if self.padding == 'same':
            out_height = np.ceil(float(height) / float(self.stride)).astype(int)
            out_width = np.ceil(float(width) / float(self.stride)).astype(int)
            padding_height = max((out_height - 1) * self.stride + self.kernel_size - height, 0)
            padding_width = max((out_width - 1) * self.stride + self.kernel_size - width, 0)
            padding_top = padding_height // 2
            padding_bottom = padding_height - padding_top
            padding_left = padding_width // 2
            padding_right = padding_width - padding_left
        elif self.padding == 'valid':
            out_height = np.ceil(float(height - self.kernel_size + 1) / float(self.stride)).astype(int)
            out_width = np.ceil(float(width - self.kernel_size + 1) / float(self.stride)).astype(int)
            padding_top = padding_bottom = padding_left = padding_right = 0
        else:
            raise ValueError("Padding must be 'same' or 'valid'.")

        input_padded = np.pad(input_data, ((0, 0), (0, 0), (padding_top, padding_bottom), (padding_left, padding_right)), mode='constant')

        out = np.zeros((batch, out_height, out_width, self.output_channels))
        
        for n in range(batch):
            for k in range(self.output_channels):
                for y in range(out_height):
                    for x in range(out_width):
                        vertical_start = y * self.stride
                        vertical_end = vertical_start + self.kernel_size
                        horizontal_start = x * self.stride
                        horizontal_end = horizontal_start + self.kernel_size
                        input_slice = input_padded[n, :, vertical_start:vertical_end, horizontal_start:horizontal_end]
                        out[n, y, x, k] = np.sum(input_slice * self.kernel[:, :, :, k]) + self.bias[k]
        return out
        return out
            
    def forward(self, input_data):
        pass
    def grad_x(self):
        pass
    def grad_kernel(self):
        pass

In [410]:
class Conv2DTrLayer(Layer):
    def __init__(self, kernel_size=3, input_channels=2, output_channels=3, 
                 padding=0, stride=1, K_init=None, b_init=None):      
        # padding: число (сколько отрезать от модифицированной входной карты)
        # Работаем с квадратными ядрами, поэтому kernel_size - одно число
        # stride - одно число (коэффициент расширения)
        # Фильтр размерности [kernel_size, kernel_size, input_channels, output_channels]
        self.name = 'Conv2DTr'
        self.kernel_size = kernel_size
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.kernel = K_init
        self.bias = b_init
        self.padding = padding
        self.stride = stride
    def forward(self, input_data):
        # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
        # Вначале нужно проверить на согласование размерностей входных данных и ядра!
        # Нужно заполнить Numpy-тензор out 
        batch, channels, height, width = input_data.shape
        
        if channels != self.input_channels:
            raise ValueError("Input channel dimension does not match kernel's channel dimension.")
        
        out_height = (height - 1) * self.stride + self.kernel_size - 2 * self.padding
        out_width = (width - 1) * self.stride + self.kernel_size - 2 * self.padding

        out = np.zeros((batch, self.output_channels, out_height, out_width))
        
        for n in range(batch):
            for k in range(self.output_channels):
                for y in range(height):
                    for x in range(width):
                        y_start = y * self.stride - self.padding
                        y_end = y_start + self.kernel_size
                        x_start = x * self.stride - self.padding
                        x_end = x_start + self.kernel_size
                        out[n, k, y_start:y_end, x_start:x_end] += input_data[n, :, y, x][None, :, None, None] * self.kernel[:, :, :, k]
        for k in range(self.output_channels):
            out[:, k, :, :] += self.bias[k]
        return out

    def forward(self, input_data):
        pass
    def grad_x(self):
        pass
    def grad_kernel(self):
        pass

#### 1.4 Теперь настало время теста. 
#### Если вы всё сделали правильно, то запустив следующие ячейки у вас должна появиться надпись: Test PASSED

Переходить к дальнейшим заданиям не имеем никакого смысла, пока вы не добьётесь прохождение теста
    

#### Чтение данных

In [411]:
import numpy as np
np.random.seed(123)
from keras.utils import to_categorical
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 1, 28, 28)
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

Y_train = to_categorical(y_train, 10)
Y_test = to_categorical(y_test, 10)
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)


(60000, 1, 28, 28) (60000, 10) (10000, 1, 28, 28) (10000, 10)


#### Подготовка моделей

In [412]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Convolution2D, Conv2D, MaxPooling2D

print(keras.__version__)

def get_keras_model():
    input_image = Input(shape=(1, 28, 28))
    pool1 = MaxPooling2D(pool_size=(2,2), data_format='channels_first')(input_image)
    flatten = Flatten()(pool1)
    dense1 = Dense(10, activation='softmax')(flatten)
    model = Model(inputs=input_image, outputs=dense1)

    from keras.optimizers import Adam, SGD
    sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])

    history = model.fit(X_train, Y_train, validation_split=0.25, 
                    batch_size=32, epochs=2, verbose=1)
    return model

3.0.5


In [413]:
def get_our_model(keras_model):
    maxpool = MaxPooling()
    flatten = FlattenLayer()
    dense = DenseLayer(196, 10, W_init=keras_model.get_weights()[0],
                       b_init=keras_model.get_weights()[1])
    softmax = Softmax()
    net = Network([maxpool, flatten, dense, softmax])
    return net

In [414]:
keras_model = get_keras_model()
our_model = get_our_model(keras_model)

Epoch 1/2
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7815 - loss: 0.8314 - val_accuracy: 0.8913 - val_loss: 0.3826
Epoch 2/2
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8919 - loss: 0.3869 - val_accuracy: 0.9005 - val_loss: 0.3448


In [415]:
keras_prediction = keras_model.predict(X_test)
our_model_prediction = our_model.predict(X_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 893us/step


In [416]:
if np.sum(np.abs(keras_prediction - our_model_prediction)) < 0.01:
    print('Test PASSED')
else:
    print('Something went wrong!')

Test PASSED


### 2. Вычисление производных по входу для слоёв нейронной сети

В данном задании запрещено использовать численные формулы для вычисления производных.

#### 2.1  Реализуйте метод forward для класса CrossEntropy
Напоминание: $$ crossentropy = L(p, y) =  - \sum\limits_i y_i log p_i, $$
где вектор $(p_1, ..., p_k) $ -  выход классификационного алгоритма, а $(y_1,..., y_k)$ - правильные метки класса в унарной кодировке (one-hot encoding)

In [417]:
class CrossEntropy(object):
    def __init__(self, eps=0.00001):
        self.name = 'CrossEntropy'
        self.eps = eps
    
    def forward(self, input_data, labels):
        input_data = np.log(input_data)
        out = np.zeros(input_data.shape[0])
        for i in range(input_data.shape[0]):
            out[i] = np.dot(input_data[i, :], labels[i, :])
        return out
    
    def calculate_loss(self,input_data, labels):
        return self.forward(input_data, labels)
    
    def grad_x(self, input_data, labels):
        out = np.zeros((input_data.shape[0], input_data.shape[1]))
        for i in range(out.shape[0]):
            for j in range(out.shape[1]):
                out[i, j] = labels[i, j] / input_data[i, j]
        return out

#### 2.2  Реализуйте метод grad_x класса CrossEntropy, который возвращает $\frac{\partial L}{\partial p}$

Проверить работоспособность кода поможет следующий тест:

In [418]:
def numerical_diff_net(net, x, labels):
    eps = 0.00001
    right_answer = []
    for i in range(len(x[0])):
        delta = np.zeros(len(x[0]))
        delta[i] = eps
        diff = (net.calculate_loss(x + delta, labels) - net.calculate_loss(x-delta, labels)) / (2*eps)
        right_answer.append(diff)
    return np.array(right_answer).T

def test_net(net):
    x = np.array([[1, 2, 3], [2, 3, 4]])
    labels = np.array([[0.3, 0.2, 0.5], [0.3, 0.2, 0.5]])
    num_grad = numerical_diff_net(net, x, labels)
    grad = net.grad_x(x, labels)
    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print('Test PASSED')
    else:
        print('Something went wrong!')
        print('Numerical grad is')
        print(num_grad)
        print('Your gradiend is ')
        print(grad)
        
loss = CrossEntropy()
test_net(loss)

Test PASSED


#### 2.3  Реализуйте метод grad_x класса Softmax, который возвращает $\frac{\partial Softmax}{\partial x}$

Проверить работоспособность кода поможет следующий тест:

In [419]:
def numerical_diff_layer(layer, x):
    eps = 0.00001
    right_answer = []
    for i in range(len(x[0])):
        delta = np.zeros(len(x[0]))
        delta[i] = eps
        diff = (layer.forward(x + delta) - layer.forward(x-delta)) / (2*eps)
        right_answer.append(diff.T)
    return np.array(right_answer).T

def test_layer(layer):
    x = np.array([[1, 2, 3], [2, -3, 4]])
    num_grad = numerical_diff_layer(layer, x)
    grad = layer.grad_x(x)
    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print('Test PASSED')
    else:
        print('Something went wrong!')
        print('Numerical grad is')
        print(num_grad)
        print('Your gradiend is ')
        print(grad)
        
layer = Softmax()
test_layer(layer)

Test PASSED


#### 2.4  Реализуйте метод grad_x для классов ReLU и DenseLayer

In [420]:
layer = ReLU()
test_layer(layer)

Test PASSED


In [421]:
layer = DenseLayer(3,4)
test_layer(layer)

Test PASSED


#### 2.5 (4 балла) Для класса Network реализуйте метод grad_x, который должен реализовывать взятие производной от лосса по входу

In [422]:
net = Network([DenseLayer(3, 10), ReLU(), DenseLayer(10, 3), Softmax()], loss=CrossEntropy())
test_net(net)

Test PASSED


### 3. Реализация градиентов по параметрам и метода обратного распространения ошибки с обновлением парметров сети

#### 3.1  Реализуйте функции grad_b и grad_W. При подготовке теста grad_W предполагается, что W является отномерным вектором.

In [423]:
def numerical_grad_b(input_size, output_size, b, W, x):
    eps = 0.00001
    right_answer = []
    for i in range(len(b)):
        delta = np.zeros(b.shape)
        delta[i] = eps
        dense1 = DenseLayer(input_size, output_size, W_init=W, b_init=b+delta)
        dense2 = DenseLayer(input_size, output_size, W_init=W, b_init=b-delta)
        diff = (dense1.forward(x) - dense2.forward(x)) / (2*eps)
        right_answer.append(diff.T)
    return np.array(right_answer).T

def test_grad_b():
    input_size = 3
    output_size = 4 
    W_init = np.random.random((input_size, output_size))
    b_init = np.random.random((output_size,))
    x = np.random.random((2, input_size))
    
    dense = DenseLayer(input_size, output_size, W_init, b_init)
    grad = dense.grad_b(x)

    num_grad = numerical_grad_b(input_size, output_size, b_init, W_init, x)
    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print('Test PASSED')
    else:
        print('Something went wrong!')
        print('Numerical grad is')
        print(num_grad)
        print('Your gradiend is ')
        print(grad)

test_grad_b()

Test PASSED


In [429]:
def numerical_grad_W(input_size, output_size, b, W, x):
    eps = 0.00001
    grad_W = np.zeros((x.shape[0], output_size, input_size))  # Correct shape
    for i in range(W.shape[0]):
        for j in range(W.shape[1]):
            delta = np.zeros(W.shape)
            delta[i, j] = eps
            dense1 = DenseLayer(input_size, output_size, W_init=W+delta, b_init=b)
            dense2 = DenseLayer(input_size, output_size, W_init=W-delta, b_init=b)
            diff = (dense1.forward(x) - dense2.forward(x)) / (2*eps)
            grad_W[:, j, i] = diff[:, j]  # Correct assignment
    return grad_W

def test_grad_W():
    input_size = 3
    output_size = 4 
    W_init = np.random.random((input_size, output_size))
    b_init = np.random.random(output_size)
    x = np.random.random((2, input_size))
        
    dense = DenseLayer(input_size, output_size, W_init, b_init)
    grad = dense.grad_W(x)

    num_grad = numerical_grad_W(input_size, output_size, b_init, W_init, x)

    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print('Test PASSED')
    else:
        print('Something went wrong!')
        print('Numerical grad is')
        print(num_grad)
        print('Your gradient is ')
        print(grad)

test_grad_W()

Test PASSED


#### 3.2 Полностью реализуйте метод обратного распространения ошибки в функции train_step класса Network


Рекомендуем реализовать сначала функцию Network.grad_param(), которая возвращает список длиной в количество слоёв и элементом которого является список градиентов по параметрам.
После чего, имея список градиентов, написать функцию обновления параметров для каждого слоя. 

Совет: рекомендуем написать тест для кода подсчета градиента по параметрам, чтобы быть уверенным в том, что градиент через всю сеть считается правильно
    

#### 3.3 Ознакомьтесь с реализацией функции fit класса Network. Запустите обучение модели. Если всё работает правильно, то точность на валидации должна будет возрастать

In [430]:
net = Network([DenseLayer(784, 10), Softmax()], loss=CrossEntropy())
trainX = X_train.reshape(len(X_train), -1)
net.fit(trainX[::3], Y_train[::3], validation_split=0.25, 
            batch_size=16, nb_epoch=5, learning_rate=0.01)

100%|███████████████████████████████████████| 937/937 [00:00<00:00, 1133.87it/s]


1 epoch: val 0.10


100%|███████████████████████████████████████| 937/937 [00:00<00:00, 1192.06it/s]


2 epoch: val 0.12


100%|███████████████████████████████████████| 937/937 [00:00<00:00, 1189.50it/s]


3 epoch: val 0.13


100%|███████████████████████████████████████| 937/937 [00:00<00:00, 1192.43it/s]


4 epoch: val 0.14


100%|███████████████████████████████████████| 937/937 [00:00<00:00, 1186.86it/s]


5 epoch: val 0.14


In [431]:
net = Network([DenseLayer(784, 20), ReLU(), DenseLayer(20, 10), Softmax()], loss=CrossEntropy())
trainX = X_train.reshape(len(X_train), -1)
net.fit(trainX[::6], Y_train[::6], validation_split=0.25, 
            batch_size=16, nb_epoch=5, learning_rate=0.001)    

100%|████████████████████████████████████████| 468/468 [00:01<00:00, 425.70it/s]


1 epoch: val 0.09


100%|████████████████████████████████████████| 468/468 [00:01<00:00, 440.81it/s]


2 epoch: val 0.09


100%|████████████████████████████████████████| 468/468 [00:01<00:00, 441.14it/s]


3 epoch: val 0.09


100%|████████████████████████████████████████| 468/468 [00:01<00:00, 365.03it/s]


4 epoch: val 0.09


100%|████████████████████████████████████████| 468/468 [00:01<00:00, 411.55it/s]


5 epoch: val 0.09


#### 3.5 Продемонстрируйте, что ваша реализация позволяет обучать более глубокие нейронные сети 